In [44]:
import pandas as pd
import tensorflow as tf


In [45]:
traffic_df = pd.read_csv('Traffic.csv')
traffic_df.head()

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal


In [57]:
traffic_df.describe()

,Time,Date,CarCount,BikeCount,BusCount,TruckCount,Total
count,2976.000000,2976.000000,2976.000000,2976.000000,2976.000000,2976.000000,2976.000000
mean,24750.000000,16.000000,68.696573,14.917339,15.279570,15.324933,114.218414
std,12470.154487,8.945775,45.850693,12.847518,14.341986,10.603833,60.190627
min,3600.000000,1.000000,6.000000,0.000000,0.000000,0.000000,21.000000
25%,14175.000000,8.000000,19.000000,5.000000,1.000000,6.000000,55.000000
50%,24750.000000,16.000000,64.000000,12.000000,12.000000,14.000000,109.000000
75%,35325.000000,24.000000,107.000000,22.000000,25.000000,23.000000,164.000000
max,45900.000000,31.000000,180.000000,70.000000,50.000000,40.000000,279.000000


In [48]:
traffic_df.columns

Index(['Time', 'Date', 'Day of the week', 'CarCount', 'BikeCount', 'BusCount',
       'TruckCount', 'Total', 'Traffic Situation'],
      dtype='object')

In [46]:
traffic_df.info

<bound method DataFrame.info of              Time  Date Day of the week  CarCount  BikeCount  BusCount  \
0     12:00:00 AM    10         Tuesday        31          0         4   
1     12:15:00 AM    10         Tuesday        49          0         3   
2     12:30:00 AM    10         Tuesday        46          0         3   
3     12:45:00 AM    10         Tuesday        51          0         2   
4      1:00:00 AM    10         Tuesday        57          6        15   
...           ...   ...             ...       ...        ...       ...   
2971  10:45:00 PM     9        Thursday        16          3         1   
2972  11:00:00 PM     9        Thursday        11          0         1   
2973  11:15:00 PM     9        Thursday        15          4         1   
2974  11:30:00 PM     9        Thursday        16          5         0   
2975  11:45:00 PM     9        Thursday        14          3         1   

      TruckCount  Total Traffic Situation  
0              4     39            

In [56]:
traffic_df.dtypes

Time                  int32
Date                  int64
Day of the week      object
CarCount              int64
BikeCount             int64
BusCount              int64
TruckCount            int64
Total                 int64
Traffic Situation    object
midday               object
dtype: object

In [49]:
traffic_df['midday'] = ''  # Create an empty 'midday' column

for i in range(len(traffic_df['Time'])):
    
    if traffic_df['Time'][i][-2:] == 'AM':
        traffic_df.loc[i, 'midday'] = 'AM'
        
    elif traffic_df['Time'][i][-2:] == 'PM':
        traffic_df.loc[i, 'midday'] = 'PM'
        
# removing 'AM' or 'PM' form Time column        
traffic_df['Time'] = traffic_df['Time'].str[:-2]

In [50]:
traffic_df

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation,midday
0,12:00:00,10,Tuesday,31,0,4,4,39,low,AM
1,12:15:00,10,Tuesday,49,0,3,3,55,low,AM
2,12:30:00,10,Tuesday,46,0,3,6,55,low,AM
3,12:45:00,10,Tuesday,51,0,2,5,58,low,AM
4,1:00:00,10,Tuesday,57,6,15,16,94,normal,AM
...,...,...,...,...,...,...,...,...,...,...
2971,10:45:00,9,Thursday,16,3,1,36,56,normal,PM
2972,11:00:00,9,Thursday,11,0,1,30,42,normal,PM
2973,11:15:00,9,Thursday,15,4,1,25,45,normal,PM
2974,11:30:00,9,Thursday,16,5,0,27,48,normal,PM


In [51]:
# Assuming 'Time' is the name of the column in your DataFrame
traffic_df['Time'] = pd.to_datetime(traffic_df['Time']).dt.hour * 3600 + \
                     pd.to_datetime(traffic_df['Time']).dt.minute * 60 + \
                     pd.to_datetime(traffic_df['Time']).dt.second

/tmp/ipykernel_36744/553369780.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  traffic_df['Time'] = pd.to_datetime(traffic_df['Time']).dt.hour * 3600 + \
/tmp/ipykernel_36744/553369780.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(traffic_df['Time']).dt.minute * 60 + \
/tmp/ipykernel_36744/553369780.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(traffic_df['Time']).dt.second


In [62]:
from sklearn.preprocessing import MinMaxScaler

# Select the numeric columns for normalization
numeric_columns = ['Time', 'Date', 'CarCount', 'BikeCount', 'BusCount', 'TruckCount', 'Total']

# Create a DataFrame with only the numeric columns
numeric_data = traffic_df[numeric_columns]

# Initialize the MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to the numeric data
scaler.fit(numeric_data)

# Perform the transformation
normalized_data = scaler.transform(numeric_data)

# Create a new DataFrame with the normalized data
normalized_df = pd.DataFrame(normalized_data, columns=numeric_columns)

# Concatenate the normalized numeric columns with the categorical columns
normalized_dataset = pd.concat([normalized_df, traffic_df[['Day of the week', 'Traffic Situation', 'midday']]], axis=1)


In [63]:
normalized_dataset

,Time,Date,CarCount,BikeCount,BusCount,TruckCount,Total,Day of the week,Traffic Situation,midday
0,0.936170,0.300000,0.143678,0.000000,0.08,0.100,0.069767,Tuesday,low,AM
1,0.957447,0.300000,0.247126,0.000000,0.06,0.075,0.131783,Tuesday,low,AM
2,0.978723,0.300000,0.229885,0.000000,0.06,0.150,0.131783,Tuesday,low,AM
3,1.000000,0.300000,0.258621,0.000000,0.04,0.125,0.143411,Tuesday,low,AM
4,0.000000,0.300000,0.293103,0.085714,0.30,0.400,0.282946,Tuesday,normal,AM
...,...,...,...,...,...,...,...,...,...,...
2971,0.829787,0.266667,0.057471,0.042857,0.02,0.900,0.135659,Thursday,normal,PM
2972,0.851064,0.266667,0.028736,0.000000,0.02,0.750,0.081395,Thursday,normal,PM
2973,0.872340,0.266667,0.051724,0.057143,0.02,0.625,0.093023,Thursday,normal,PM
2974,0.893617,0.266667,0.057471,0.071429,0.00,0.675,0.104651,Thursday,normal,PM


In [67]:
# Assuming 'dataset' is the name of your DataFrame

# Perform one-hot encoding on 'Day of the week' column
day_of_week_encoded = pd.get_dummies(traffic_df['Day of the week'], prefix='day')

# Perform one-hot encoding on 'Traffic Situation' column
traffic_situation_encoded = pd.get_dummies(traffic_df['Traffic Situation'], prefix='traffic')

# Perform one-hot encoding on 'midday' column
midday_encoded = pd.get_dummies(traffic_df['midday'], prefix='midday')

# Concatenate the encoded columns with the original DataFrame
dataset = pd.concat([traffic_df.drop(['Day of the week', 'Traffic Situation', 'midday'], axis=1), 
                             day_of_week_encoded, traffic_situation_encoded, midday_encoded], axis=1)

In [68]:
dataset

,Time,Date,CarCount,BikeCount,BusCount,TruckCount,Total,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,traffic_heavy,traffic_high,traffic_low,traffic_normal,midday_AM,midday_PM
0,43200,10,31,0,4,4,39,False,False,False,False,False,True,False,False,False,True,False,True,False
1,44100,10,49,0,3,3,55,False,False,False,False,False,True,False,False,False,True,False,True,False
2,45000,10,46,0,3,6,55,False,False,False,False,False,True,False,False,False,True,False,True,False
3,45900,10,51,0,2,5,58,False,False,False,False,False,True,False,False,False,True,False,True,False
4,3600,10,57,6,15,16,94,False,False,False,False,False,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2971,38700,9,16,3,1,36,56,False,False,False,False,True,False,False,False,False,False,True,False,True
2972,39600,9,11,0,1,30,42,False,False,False,False,True,False,False,False,False,False,True,False,True
2973,40500,9,15,4,1,25,45,False,False,False,False,True,False,False,False,False,False,True,False,True
2974,41400,9,16,5,0,27,48,False,False,False,False,True,False,False,False,False,False,True,False,True


In [ ]:
mapping = {False: 0, True: 1}

# Map the values using the mapping dictionary and convert them to integers
dataset['target'] = dataset['target'].map(mapping).astype(int)